In [395]:
class User:
    def __init__(self, name, label="none"): 
        self.id = name
        self.label = label
        self.advisors = [] #neighbors
        self.providers = []
        self.interactions = {}
        self.total_interactions = 0
        self.provider_raputation = {}
        self.advisor_reputation = {}
        self.decay = 1 #longitivity
        self.phi = 0.5 #w
        self.epsilon = 1 #uncertainity
        self.limit = 0 #tolarance limit
        self.alpha = 0.8 #consumer confidence
        self.satisfaction = {}
        self.disatisfaction = {}
        self.b_satisfaction = {}
        self.b_disatisfaction = {}
            
    def interact(self, provider, rating):
        if provider.id != self.id:
            if provider in self.providers:
                ratelist = self.interactions[provider]
            else:
                ratelist = []
                self.providers.append(provider)

            ratelist.append(rating)
            self.interactions[provider] = ratelist
            self.total_interactions += 1

            print("Consumer :"+str(self.id)+" rated Provider :"+str(provider.id)+" with "+str(rating))
#             self.eval_provider(provider, rating)
#             provider.eval_provider(provider, rating, bayes = True)
        else:
            print('I cant interact with myself')
        
        
    def eval_provider(self, provider, rating, bayes = False):
        exp_val_n = len(self.interactions[provider])

        rj = (1- rating) * self.phi #disatisfaction self.phi = w, rj = ds
        sj = (1+ rating) * self.phi #satisfaction self.phi = w  sj = sj
        
        if bayes:
            print("Bayes Consumer :"+str(self.id)+" evaluates Provider :"+str(provider.id)+" satisfaction: "+str(sj)+ "disatisfaction :"+str(rj))
            if provider in self.b_satisfaction:
                cumilative_satisfaction = self.b_satisfaction[provider]
                cumilative_disatisfaction = self.b_disatisfaction[provider]
                cumilative_satisfaction += sj
                cumilative_disatisfaction += rj
            else:
                cumilative_satisfaction = sj
                cumilative_disatisfaction = rj
            self.b_satisfaction[provider] = cumilative_satisfaction
            self.b_disatisfaction[provider] = cumilative_disatisfaction
        else:
            decay_factor = self.decay **(exp_val_n - self.limit)
            rj *= decay_factor
            sj *= decay_factor
            self.limit += 1
            print("Consumer :"+str(self.id)+" evaluates Provider :"+str(provider.id)+" satisfaction: "+str(sj)+ "disatisfaction :"+str(rj))
            if provider in self.satisfaction:
                cumilative_satisfaction = self.satisfaction[provider]
                cumilative_disatisfaction = self.disatisfaction[provider]
                cumilative_satisfaction += sj
                cumilative_disatisfaction +=rj
            else:
                cumilative_satisfaction = sj
                cumilative_disatisfaction = rj
            self.satisfaction[provider] = cumilative_satisfaction
            self.disatisfaction[provider] = cumilative_disatisfaction
        
        print("Consumer :"+str(self.id)+"'s current satisfaction and disatisfaction for Provider :"+str(provider.id)+" are : "+str(cumilative_satisfaction)+ " :"+str(cumilative_disatisfaction))
        self.get_belief(provider)
    
    def get_belief(self, provider):
            cum_sat = self.satisfaction[provider]
            cum_dis = self.disatisfaction[provider]
            total_sat_dit = cum_dis+cum_sat
            den = total_sat_dit + self.epsilon 
            print(total_sat_dit)
            belief = cum_sat /(den)
            disbelief = cum_dis /(den)
            uncertainity = self.epsilon / (den)
            
#             print("Consumer :"+str(self.id)+"'s current belief,disbelief, and uncertainity for Provider :"+str(provider.id)+" are : "+str(belief)+ " :"+str(disbelief)+ " : "+str(uncertainity))
#             print("Sanitycheck add all :"+str((belief + disbelief + uncertainity)))
        
            return belief, disbelief, uncertainity
        
        
    def provider_raputation(self, provider):
        return provider_raputation[provider]
    
    def get_providers(self):
        return self.providers
        

    def get_similarity(self, advisor):

        
        common_providers = [x for x in advisor.get_providers() if x in self.get_providers()]
        similarity = -100 #note this is just for checking if the advisor has a common vendor
        
        if len(common_providers) > 0:
            cum_dist = 0
            for p in common_providers:
                    
                    a_bel, a_disbel, a_unc = self.get_belief(p)
                    b_bel, b_disbel, b_unc = advisor.get_belief(p)
                    nonsqrt= self.get_distance(advisor, p)
                    dist = nonsqrt** (1/2)
                    dist = dist/(2**(1/2))
                    cum_dist += dist
            meanormilsed_distance = cum_dist/len(common_providers)
            similarity = 1 - meanormilsed_distance
#             print("Consumer :"+str(self.id)+" similarity with advisor :"+str(advisor.id)+" is "+str(similarity)+", nmd is "+str(meanormilsed_distance))
        return similarity
    
    def get_distance(self, advisor, p):
        a_bel, a_disbel, a_unc = self.get_belief(p)
        b_bel, b_disbel, b_unc = advisor.get_belief(p)
        nonsqrt=((a_bel - b_bel)**2) + ((a_disbel - b_disbel)**2) + ((a_unc - b_unc)**2)
#         print('ED('+str(p.id)+'_,'+str(self.id)+') = '+str(nonsqrt))
        return nonsqrt
            
    def get_global_crediblity(self):
        cum_bel = 0;
        for p in self.providers:
            b ,_ , _ = p.get_belief(self)
            cum_bel += b
        return cum_bel/len(self.providers)
            
    
    def set_adivsors(self, neigbors):
        self.advisors = neigbors
    def add_an_adivsor(self, neigbor):
        self.advisors.append(neigbor)
    
    def get_id(self):
        self.id
    

    def my_bayes(self, p):
        sat = 0
        dis = 0
        if p in self.b_satisfaction:
            sat = self.b_satisfaction[p]
            dis = self.b_disatisfaction[p]
        return sat, dis
    

    
    def p_g_reputation(self, provider, all_advisors, filter_threshold = 0.5):
        sim_count = 0
        gc_count = 0
        p_rep = 0
        g_rep = 0
        for a in all_advisors:
            if a.id != self.id:
                ap_bel, _, _ = a.get_belief(provider)
                sim = self.get_similarity(a)
                if sim != -100: #meaning have no common vendos
                    sim_count += 1
                    if sim > filter_threshold:
                        p_rep += (sim * ap_bel)
                else:
                    gc = a.get_global_crediblity()
                    g_rep += (gc * ap_bel)
        print('personilsed_reputaion: '+str(p_rep)+' g_rep: '+str(g_rep))
        reputation = 0
        
        if sim_count > 0:
            reputation = (self.alpha * p_rep)/sim_count

        if (len(all_advisors) - sim_count) > 0:
            reputation += ((1 - self.alpha) * g_rep)/(len(all_advisors) - sim_count)
        return reputation
        
    
    def init_trust(provider, beta=0.5, trust_propensity=0.5):
        rep = self.reputation(provider)
        return (beta * trust_propensity ) + ((1-beta) * rep)
    
    def trust_T(df,initT, beta=0.5):
        t = self.init_trust(provider)
        b, d, u = self.get_belief(provider)
        trust = b + (t * u)
        trust_dash = d + ((1 - t) * u)
        print("Sum test ="+str(trust + trust_dash))
        return trust
        

In [396]:
class Consumer(User):
    def __init__(self, name, label="C"):
        User.__init__(self,name, label)
        

class Producer(User):
    def __init__(self, name, label="P"):
        User.__init__(self,name, label)

# class Advisor(Users):
#     def __init__(self, name):
#         Users.__init__(self,name)

In [397]:
l_consumers = {}
l_producers = {}
consumer_id_start_range = 100
producer_id_start_range = 200
for x in range(11):
    print(consumer_id_start_range+x)
    name ='C'+str(x+1)
    c = Consumer(consumer_id_start_range+x, name)
    l_consumers[consumer_id_start_range+x] = c
    if x % 2 == 0:
        print(producer_id_start_range+x)
        p = Producer(producer_id_start_range+x)
        l_producers[producer_id_start_range+x] = p
    

100
200
101
102
202
103
104
204
105
106
206
107
108
208
109
110
210


In [398]:
len(l_consumers)

11

In [399]:
def rategen(goodn, badn, co, pr):  
    for g in range(goodn):
        print('========Goood loop===='+str(g+1))
        co.interact(pr, -1)
        co.eval_provider(pr, -1)
        co.eval_provider(pr, -1, bayes=True)

        pr.interact(co, 1)
        pr.eval_provider(co, 1)
        pr.eval_provider(co, 1, bayes=True)

    for i in range(badn):
        print('========Bad loop===='+str(i+1))
        co.interact(pr, -1)
        co.eval_provider(pr, -1)
        co.eval_provider(pr, -1, bayes=True)

        pr.interact(co, -1)
        pr.eval_provider(co, -1)
        pr.eval_provider(co, -1, bayes=True)

In [400]:
c1 = l_consumers[100]
x = l_producers[200]

for i in range(15):
    print('======i count'+str(i+1))
    c1.interact(x, 1)
    c1.eval_provider(x, 1)
    c1.eval_provider(x, 1, bayes=True)
    
    x.interact(c1, 1)
    x.eval_provider(c1, 1)
    x.eval_provider(c1, 1, bayes=True)

======i count1
Consumer :100 rated Provider :200 with 1
Consumer :100 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :100's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Bayes Consumer :100 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :100's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Consumer :200 rated Provider :100 with 1
Consumer :200 evaluates Provider :100 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :100 are : 1.0 :0.0
1.0
Bayes Consumer :200 evaluates Provider :100 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :100 are : 1.0 :0.0
1.0
======i count2
Consumer :100 rated Provider :200 with 1
Consumer :100 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :100's current satisfaction and disatisfaction for Provider :200 are : 2.0 :0.0

In [401]:
c1 = l_consumers[100]
y = l_producers[202]
rategen(1, 9, c1, y)

========Goood loop====1
Consumer :100 rated Provider :202 with -1
Consumer :100 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider :202 are : 0.0 :1.0
1.0
Bayes Consumer :100 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider :202 are : 0.0 :1.0
1.0
Consumer :202 rated Provider :100 with 1
Consumer :202 evaluates Provider :100 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :100 are : 1.0 :0.0
1.0
Bayes Consumer :202 evaluates Provider :100 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :100 are : 1.0 :0.0
1.0
========Bad loop====1
Consumer :100 rated Provider :202 with -1
Consumer :100 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider :

In [402]:
z = l_producers[204]
rategen(2, 8, c1, z)

========Goood loop====1
Consumer :100 rated Provider :204 with -1
Consumer :100 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider :204 are : 0.0 :1.0
1.0
Bayes Consumer :100 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider :204 are : 0.0 :1.0
1.0
Consumer :204 rated Provider :100 with 1
Consumer :204 evaluates Provider :100 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :100 are : 1.0 :0.0
1.0
Bayes Consumer :204 evaluates Provider :100 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :100 are : 1.0 :0.0
1.0
========Goood loop====2
Consumer :100 rated Provider :204 with -1
Consumer :100 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider

In [403]:
u = l_producers[206]
rategen(1, 9, c1, u)

========Goood loop====1
Consumer :100 rated Provider :206 with -1
Consumer :100 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider :206 are : 0.0 :1.0
1.0
Bayes Consumer :100 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider :206 are : 0.0 :1.0
1.0
Consumer :206 rated Provider :100 with 1
Consumer :206 evaluates Provider :100 satisfaction: 1.0disatisfaction :0.0
Consumer :206's current satisfaction and disatisfaction for Provider :100 are : 1.0 :0.0
1.0
Bayes Consumer :206 evaluates Provider :100 satisfaction: 1.0disatisfaction :0.0
Consumer :206's current satisfaction and disatisfaction for Provider :100 are : 1.0 :0.0
1.0
========Bad loop====1
Consumer :100 rated Provider :206 with -1
Consumer :100 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider :

In [404]:
v = l_producers[208]
rategen(2, 8, c1, v)

========Goood loop====1
Consumer :100 rated Provider :208 with -1
Consumer :100 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider :208 are : 0.0 :1.0
1.0
Bayes Consumer :100 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider :208 are : 0.0 :1.0
1.0
Consumer :208 rated Provider :100 with 1
Consumer :208 evaluates Provider :100 satisfaction: 1.0disatisfaction :0.0
Consumer :208's current satisfaction and disatisfaction for Provider :100 are : 1.0 :0.0
1.0
Bayes Consumer :208 evaluates Provider :100 satisfaction: 1.0disatisfaction :0.0
Consumer :208's current satisfaction and disatisfaction for Provider :100 are : 1.0 :0.0
1.0
========Goood loop====2
Consumer :100 rated Provider :208 with -1
Consumer :100 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :100's current satisfaction and disatisfaction for Provider

In [405]:
c2 = l_consumers[101]

for i in range(15):
    c2.interact(x, 1)
    c2.eval_provider(x, 1)
    c2.eval_provider(x, 1, bayes=True)
    
    x.interact(c2, 1)
    x.eval_provider(c2, 1)
    x.eval_provider(c2, 1, bayes=True)

Consumer :101 rated Provider :200 with 1
Consumer :101 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :101's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Bayes Consumer :101 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :101's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Consumer :200 rated Provider :101 with 1
Consumer :200 evaluates Provider :101 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :101 are : 1.0 :0.0
1.0
Bayes Consumer :200 evaluates Provider :101 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :101 are : 1.0 :0.0
1.0
Consumer :101 rated Provider :200 with 1
Consumer :101 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :101's current satisfaction and disatisfaction for Provider :200 are : 2.0 :0.0
2.0
Bayes Consumer :101 evalu

In [406]:
rategen(1, 9, c2, y)

========Goood loop====1
Consumer :101 rated Provider :202 with -1
Consumer :101 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider :202 are : 0.0 :1.0
1.0
Bayes Consumer :101 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider :202 are : 0.0 :1.0
1.0
Consumer :202 rated Provider :101 with 1
Consumer :202 evaluates Provider :101 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :101 are : 1.0 :0.0
1.0
Bayes Consumer :202 evaluates Provider :101 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :101 are : 1.0 :0.0
1.0
========Bad loop====1
Consumer :101 rated Provider :202 with -1
Consumer :101 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider :

In [407]:
rategen(1, 9, c2, z)

========Goood loop====1
Consumer :101 rated Provider :204 with -1
Consumer :101 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider :204 are : 0.0 :1.0
1.0
Bayes Consumer :101 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider :204 are : 0.0 :1.0
1.0
Consumer :204 rated Provider :101 with 1
Consumer :204 evaluates Provider :101 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :101 are : 1.0 :0.0
1.0
Bayes Consumer :204 evaluates Provider :101 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :101 are : 1.0 :0.0
1.0
========Bad loop====1
Consumer :101 rated Provider :204 with -1
Consumer :101 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider :

In [408]:
rategen(2, 8, c2, u)

========Goood loop====1
Consumer :101 rated Provider :206 with -1
Consumer :101 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider :206 are : 0.0 :1.0
1.0
Bayes Consumer :101 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider :206 are : 0.0 :1.0
1.0
Consumer :206 rated Provider :101 with 1
Consumer :206 evaluates Provider :101 satisfaction: 1.0disatisfaction :0.0
Consumer :206's current satisfaction and disatisfaction for Provider :101 are : 1.0 :0.0
1.0
Bayes Consumer :206 evaluates Provider :101 satisfaction: 1.0disatisfaction :0.0
Consumer :206's current satisfaction and disatisfaction for Provider :101 are : 1.0 :0.0
1.0
========Goood loop====2
Consumer :101 rated Provider :206 with -1
Consumer :101 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider

In [409]:
rategen(2, 8, c2, v)

========Goood loop====1
Consumer :101 rated Provider :208 with -1
Consumer :101 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider :208 are : 0.0 :1.0
1.0
Bayes Consumer :101 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider :208 are : 0.0 :1.0
1.0
Consumer :208 rated Provider :101 with 1
Consumer :208 evaluates Provider :101 satisfaction: 1.0disatisfaction :0.0
Consumer :208's current satisfaction and disatisfaction for Provider :101 are : 1.0 :0.0
1.0
Bayes Consumer :208 evaluates Provider :101 satisfaction: 1.0disatisfaction :0.0
Consumer :208's current satisfaction and disatisfaction for Provider :101 are : 1.0 :0.0
1.0
========Goood loop====2
Consumer :101 rated Provider :208 with -1
Consumer :101 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :101's current satisfaction and disatisfaction for Provider

In [410]:
# rategen(1, 9, c2, y)

In [411]:
c3 = l_consumers[102]

for i in range(15):
    c3.interact(x, 1)
    c3.eval_provider(x, 1)
    c3.eval_provider(x, 1, bayes=True)
    
    x.interact(c3, 1)
    x.eval_provider(c3, 1)
    x.eval_provider(c3, 1, bayes=True)

Consumer :102 rated Provider :200 with 1
Consumer :102 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :102's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Bayes Consumer :102 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :102's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Consumer :200 rated Provider :102 with 1
Consumer :200 evaluates Provider :102 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :102 are : 1.0 :0.0
1.0
Bayes Consumer :200 evaluates Provider :102 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :102 are : 1.0 :0.0
1.0
Consumer :102 rated Provider :200 with 1
Consumer :102 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :102's current satisfaction and disatisfaction for Provider :200 are : 2.0 :0.0
2.0
Bayes Consumer :102 evalu

In [412]:
rategen(2, 8, c3, y)

========Goood loop====1
Consumer :102 rated Provider :202 with -1
Consumer :102 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider :202 are : 0.0 :1.0
1.0
Bayes Consumer :102 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider :202 are : 0.0 :1.0
1.0
Consumer :202 rated Provider :102 with 1
Consumer :202 evaluates Provider :102 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :102 are : 1.0 :0.0
1.0
Bayes Consumer :202 evaluates Provider :102 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :102 are : 1.0 :0.0
1.0
========Goood loop====2
Consumer :102 rated Provider :202 with -1
Consumer :102 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider

In [413]:
rategen(2, 8, c3, z)

========Goood loop====1
Consumer :102 rated Provider :204 with -1
Consumer :102 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider :204 are : 0.0 :1.0
1.0
Bayes Consumer :102 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider :204 are : 0.0 :1.0
1.0
Consumer :204 rated Provider :102 with 1
Consumer :204 evaluates Provider :102 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :102 are : 1.0 :0.0
1.0
Bayes Consumer :204 evaluates Provider :102 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :102 are : 1.0 :0.0
1.0
========Goood loop====2
Consumer :102 rated Provider :204 with -1
Consumer :102 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider

In [414]:
rategen(1, 9, c3, u)

========Goood loop====1
Consumer :102 rated Provider :206 with -1
Consumer :102 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider :206 are : 0.0 :1.0
1.0
Bayes Consumer :102 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider :206 are : 0.0 :1.0
1.0
Consumer :206 rated Provider :102 with 1
Consumer :206 evaluates Provider :102 satisfaction: 1.0disatisfaction :0.0
Consumer :206's current satisfaction and disatisfaction for Provider :102 are : 1.0 :0.0
1.0
Bayes Consumer :206 evaluates Provider :102 satisfaction: 1.0disatisfaction :0.0
Consumer :206's current satisfaction and disatisfaction for Provider :102 are : 1.0 :0.0
1.0
========Bad loop====1
Consumer :102 rated Provider :206 with -1
Consumer :102 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider :

In [415]:
rategen(2, 8, c3, v)

========Goood loop====1
Consumer :102 rated Provider :208 with -1
Consumer :102 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider :208 are : 0.0 :1.0
1.0
Bayes Consumer :102 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider :208 are : 0.0 :1.0
1.0
Consumer :208 rated Provider :102 with 1
Consumer :208 evaluates Provider :102 satisfaction: 1.0disatisfaction :0.0
Consumer :208's current satisfaction and disatisfaction for Provider :102 are : 1.0 :0.0
1.0
Bayes Consumer :208 evaluates Provider :102 satisfaction: 1.0disatisfaction :0.0
Consumer :208's current satisfaction and disatisfaction for Provider :102 are : 1.0 :0.0
1.0
========Goood loop====2
Consumer :102 rated Provider :208 with -1
Consumer :102 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :102's current satisfaction and disatisfaction for Provider

In [416]:
#c4, x
c4 = l_consumers[103]

for i in range(15):
    c4.interact(x, 1)
    c4.eval_provider(x, 1)
    c4.eval_provider(x, 1, bayes=True)
    
    x.interact(c4, 1)
    x.eval_provider(c4, 1)
    x.eval_provider(c4, 1, bayes=True)

Consumer :103 rated Provider :200 with 1
Consumer :103 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :103's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Bayes Consumer :103 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :103's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Consumer :200 rated Provider :103 with 1
Consumer :200 evaluates Provider :103 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :103 are : 1.0 :0.0
1.0
Bayes Consumer :200 evaluates Provider :103 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :103 are : 1.0 :0.0
1.0
Consumer :103 rated Provider :200 with 1
Consumer :103 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :103's current satisfaction and disatisfaction for Provider :200 are : 2.0 :0.0
2.0
Bayes Consumer :103 evalu

In [417]:
rategen(1, 9, c4, y)

========Goood loop====1
Consumer :103 rated Provider :202 with -1
Consumer :103 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider :202 are : 0.0 :1.0
1.0
Bayes Consumer :103 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider :202 are : 0.0 :1.0
1.0
Consumer :202 rated Provider :103 with 1
Consumer :202 evaluates Provider :103 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :103 are : 1.0 :0.0
1.0
Bayes Consumer :202 evaluates Provider :103 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :103 are : 1.0 :0.0
1.0
========Bad loop====1
Consumer :103 rated Provider :202 with -1
Consumer :103 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider :

In [418]:
rategen(1, 9, c4, z)

========Goood loop====1
Consumer :103 rated Provider :204 with -1
Consumer :103 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider :204 are : 0.0 :1.0
1.0
Bayes Consumer :103 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider :204 are : 0.0 :1.0
1.0
Consumer :204 rated Provider :103 with 1
Consumer :204 evaluates Provider :103 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :103 are : 1.0 :0.0
1.0
Bayes Consumer :204 evaluates Provider :103 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :103 are : 1.0 :0.0
1.0
========Bad loop====1
Consumer :103 rated Provider :204 with -1
Consumer :103 evaluates Provider :204 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider :

In [419]:
rategen(1, 9, c4, u)

========Goood loop====1
Consumer :103 rated Provider :206 with -1
Consumer :103 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider :206 are : 0.0 :1.0
1.0
Bayes Consumer :103 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider :206 are : 0.0 :1.0
1.0
Consumer :206 rated Provider :103 with 1
Consumer :206 evaluates Provider :103 satisfaction: 1.0disatisfaction :0.0
Consumer :206's current satisfaction and disatisfaction for Provider :103 are : 1.0 :0.0
1.0
Bayes Consumer :206 evaluates Provider :103 satisfaction: 1.0disatisfaction :0.0
Consumer :206's current satisfaction and disatisfaction for Provider :103 are : 1.0 :0.0
1.0
========Bad loop====1
Consumer :103 rated Provider :206 with -1
Consumer :103 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider :

In [420]:
rategen(2, 8, c4, v)

========Goood loop====1
Consumer :103 rated Provider :208 with -1
Consumer :103 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider :208 are : 0.0 :1.0
1.0
Bayes Consumer :103 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider :208 are : 0.0 :1.0
1.0
Consumer :208 rated Provider :103 with 1
Consumer :208 evaluates Provider :103 satisfaction: 1.0disatisfaction :0.0
Consumer :208's current satisfaction and disatisfaction for Provider :103 are : 1.0 :0.0
1.0
Bayes Consumer :208 evaluates Provider :103 satisfaction: 1.0disatisfaction :0.0
Consumer :208's current satisfaction and disatisfaction for Provider :103 are : 1.0 :0.0
1.0
========Goood loop====2
Consumer :103 rated Provider :208 with -1
Consumer :103 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :103's current satisfaction and disatisfaction for Provider

In [421]:

c5 = l_consumers[104]

c5.interact(x, -1)
c5.eval_provider(x, -1)
c5.eval_provider(x, -1, bayes=True)

x.interact(c5, -1)
x.eval_provider(c5, -1)
x.eval_provider(c5, -1, bayes=True)

Consumer :104 rated Provider :200 with -1
Consumer :104 evaluates Provider :200 satisfaction: 0.0disatisfaction :1.0
Consumer :104's current satisfaction and disatisfaction for Provider :200 are : 0.0 :1.0
1.0
Bayes Consumer :104 evaluates Provider :200 satisfaction: 0.0disatisfaction :1.0
Consumer :104's current satisfaction and disatisfaction for Provider :200 are : 0.0 :1.0
1.0
Consumer :200 rated Provider :104 with -1
Consumer :200 evaluates Provider :104 satisfaction: 0.0disatisfaction :1.0
Consumer :200's current satisfaction and disatisfaction for Provider :104 are : 0.0 :1.0
1.0
Bayes Consumer :200 evaluates Provider :104 satisfaction: 0.0disatisfaction :1.0
Consumer :200's current satisfaction and disatisfaction for Provider :104 are : 0.0 :1.0
1.0


In [422]:
#c5,y
pos_int = 5

for i in range(6):
    if i<pos_int:
        c5.interact(y, 1)
        c5.eval_provider(y, 1)
        c5.eval_provider(y, 1, bayes=True)

    else:
        c5.interact(y, -1)
        c5.eval_provider(y, -1)
        c5.eval_provider(y, -1, bayes=True)

    y.interact(c5, 1)
    y.eval_provider(c5, 1)
    y.eval_provider(c5, 1, bayes=True)
        


Consumer :104 rated Provider :202 with 1
Consumer :104 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :104's current satisfaction and disatisfaction for Provider :202 are : 1.0 :0.0
1.0
Bayes Consumer :104 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :104's current satisfaction and disatisfaction for Provider :202 are : 1.0 :0.0
1.0
Consumer :202 rated Provider :104 with 1
Consumer :202 evaluates Provider :104 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :104 are : 1.0 :0.0
1.0
Bayes Consumer :202 evaluates Provider :104 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :104 are : 1.0 :0.0
1.0
Consumer :104 rated Provider :202 with 1
Consumer :104 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :104's current satisfaction and disatisfaction for Provider :202 are : 2.0 :0.0
2.0
Bayes Consumer :104 evalu

In [423]:
#c5,z
for i in range(3):
    c5.interact(z, 1)
    c5.eval_provider(z, 1)
    c5.eval_provider(z, 1, bayes=True)

    z.interact(c5, 1)
    z.eval_provider(c5, 1)
    z.eval_provider(c5, 1, bayes=True)


c5.interact(z, -1)
c5.eval_provider(z, -1)
c5.eval_provider(z, -1, bayes=True)

z.interact(c5, 1)
z.eval_provider(c5, 1)
z.eval_provider(c5, 1, bayes=True)

c5.interact(z, -1)
c5.eval_provider(z, -1)
c5.eval_provider(z, -1, bayes=True)

z.interact(c5, -1)
z.eval_provider(c5, -1)
z.eval_provider(c5, -1, bayes=True)

Consumer :104 rated Provider :204 with 1
Consumer :104 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :104's current satisfaction and disatisfaction for Provider :204 are : 1.0 :0.0
1.0
Bayes Consumer :104 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :104's current satisfaction and disatisfaction for Provider :204 are : 1.0 :0.0
1.0
Consumer :204 rated Provider :104 with 1
Consumer :204 evaluates Provider :104 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :104 are : 1.0 :0.0
1.0
Bayes Consumer :204 evaluates Provider :104 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :104 are : 1.0 :0.0
1.0
Consumer :104 rated Provider :204 with 1
Consumer :104 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :104's current satisfaction and disatisfaction for Provider :204 are : 2.0 :0.0
2.0
Bayes Consumer :104 evalu

In [424]:
c6 = l_consumers[105]

In [425]:
#c6,x
for i in range(2):
    c6.interact(x, -1)
    c6.eval_provider(x, -1)
    c6.eval_provider(x, -1, bayes=True)

    x.interact(c6, -1)
    x.eval_provider(c6, -1)
    x.eval_provider(c6, -1, bayes=True)

Consumer :105 rated Provider :200 with -1
Consumer :105 evaluates Provider :200 satisfaction: 0.0disatisfaction :1.0
Consumer :105's current satisfaction and disatisfaction for Provider :200 are : 0.0 :1.0
1.0
Bayes Consumer :105 evaluates Provider :200 satisfaction: 0.0disatisfaction :1.0
Consumer :105's current satisfaction and disatisfaction for Provider :200 are : 0.0 :1.0
1.0
Consumer :200 rated Provider :105 with -1
Consumer :200 evaluates Provider :105 satisfaction: 0.0disatisfaction :1.0
Consumer :200's current satisfaction and disatisfaction for Provider :105 are : 0.0 :1.0
1.0
Bayes Consumer :200 evaluates Provider :105 satisfaction: 0.0disatisfaction :1.0
Consumer :200's current satisfaction and disatisfaction for Provider :105 are : 0.0 :1.0
1.0
Consumer :105 rated Provider :200 with -1
Consumer :105 evaluates Provider :200 satisfaction: 0.0disatisfaction :1.0
Consumer :105's current satisfaction and disatisfaction for Provider :200 are : 0.0 :2.0
2.0
Bayes Consumer :105 ev

In [426]:
#c6,y
for i in range(6):
    c6.interact(y, 1)
    c6.eval_provider(y, 1)
    c6.eval_provider(y, 1, bayes=True)

    y.interact(c6, 1)
    y.eval_provider(c6, 1)
    y.eval_provider(c6, 1, bayes=True)

Consumer :105 rated Provider :202 with 1
Consumer :105 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :105's current satisfaction and disatisfaction for Provider :202 are : 1.0 :0.0
1.0
Bayes Consumer :105 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :105's current satisfaction and disatisfaction for Provider :202 are : 1.0 :0.0
1.0
Consumer :202 rated Provider :105 with 1
Consumer :202 evaluates Provider :105 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :105 are : 1.0 :0.0
1.0
Bayes Consumer :202 evaluates Provider :105 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :105 are : 1.0 :0.0
1.0
Consumer :105 rated Provider :202 with 1
Consumer :105 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :105's current satisfaction and disatisfaction for Provider :202 are : 2.0 :0.0
2.0
Bayes Consumer :105 evalu

In [427]:
#c6,z
for i in range(2):
    c6.interact(z, 1)
    c6.eval_provider(z, 1)
    c6.eval_provider(z, 1, bayes=True)

    z.interact(c6, 1)
    z.eval_provider(c6, 1)
    z.eval_provider(c6, 1, bayes=True)

c6.interact(z, -1)
c6.eval_provider(z, -1)
c6.eval_provider(z, -1, bayes=True)

z.interact(c6, 1)
z.eval_provider(c6, 1)
z.eval_provider(c6, 1, bayes=True)

c6.interact(z, -1)
c6.eval_provider(z, -1)
c6.eval_provider(z, -1, bayes=True)

z.interact(c6, -1)
z.eval_provider(c6, -1)
z.eval_provider(c6, -1, bayes=True)

Consumer :105 rated Provider :204 with 1
Consumer :105 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :105's current satisfaction and disatisfaction for Provider :204 are : 1.0 :0.0
1.0
Bayes Consumer :105 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :105's current satisfaction and disatisfaction for Provider :204 are : 1.0 :0.0
1.0
Consumer :204 rated Provider :105 with 1
Consumer :204 evaluates Provider :105 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :105 are : 1.0 :0.0
1.0
Bayes Consumer :204 evaluates Provider :105 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :105 are : 1.0 :0.0
1.0
Consumer :105 rated Provider :204 with 1
Consumer :105 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :105's current satisfaction and disatisfaction for Provider :204 are : 2.0 :0.0
2.0
Bayes Consumer :105 evalu

In [428]:
c7 = l_consumers[106]

In [429]:
#c7, x
for i in range(3):
    c7.interact(x, -1)
    c7.eval_provider(x, -1)
    c7.eval_provider(x, -1, bayes=True)

    x.interact(c7, -1)
    x.eval_provider(c7, -1)
    x.eval_provider(c7, -1, bayes=True)

Consumer :106 rated Provider :200 with -1
Consumer :106 evaluates Provider :200 satisfaction: 0.0disatisfaction :1.0
Consumer :106's current satisfaction and disatisfaction for Provider :200 are : 0.0 :1.0
1.0
Bayes Consumer :106 evaluates Provider :200 satisfaction: 0.0disatisfaction :1.0
Consumer :106's current satisfaction and disatisfaction for Provider :200 are : 0.0 :1.0
1.0
Consumer :200 rated Provider :106 with -1
Consumer :200 evaluates Provider :106 satisfaction: 0.0disatisfaction :1.0
Consumer :200's current satisfaction and disatisfaction for Provider :106 are : 0.0 :1.0
1.0
Bayes Consumer :200 evaluates Provider :106 satisfaction: 0.0disatisfaction :1.0
Consumer :200's current satisfaction and disatisfaction for Provider :106 are : 0.0 :1.0
1.0
Consumer :106 rated Provider :200 with -1
Consumer :106 evaluates Provider :200 satisfaction: 0.0disatisfaction :1.0
Consumer :106's current satisfaction and disatisfaction for Provider :200 are : 0.0 :2.0
2.0
Bayes Consumer :106 ev

In [430]:
for i in range(9):
    c7.interact(y, 1)
    c7.eval_provider(y, 1)
    c7.eval_provider(y, 1, bayes=True)

    y.interact(c7, 1)
    y.eval_provider(c7, 1)
    y.eval_provider(c7, 1, bayes=True)

for i in range(4):
    c7.interact(y, -1)
    c7.eval_provider(y, -1)
    c7.eval_provider(y, -1, bayes=True)

    y.interact(c7, -1)
    y.eval_provider(c7, -1)
    y.eval_provider(c7, -1, bayes=True)
    
c7.interact(y, -1)
c7.eval_provider(y, -1)
c7.eval_provider(y, -1, bayes=True)

y.interact(c7, 1)
y.eval_provider(c7, 1)
y.eval_provider(c7, 1, bayes=True)

Consumer :106 rated Provider :202 with 1
Consumer :106 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :106's current satisfaction and disatisfaction for Provider :202 are : 1.0 :0.0
1.0
Bayes Consumer :106 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :106's current satisfaction and disatisfaction for Provider :202 are : 1.0 :0.0
1.0
Consumer :202 rated Provider :106 with 1
Consumer :202 evaluates Provider :106 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :106 are : 1.0 :0.0
1.0
Bayes Consumer :202 evaluates Provider :106 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :106 are : 1.0 :0.0
1.0
Consumer :106 rated Provider :202 with 1
Consumer :106 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :106's current satisfaction and disatisfaction for Provider :202 are : 2.0 :0.0
2.0
Bayes Consumer :106 evalu

Bayes Consumer :106 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :106's current satisfaction and disatisfaction for Provider :202 are : 9.0 :3.0
12.0
Consumer :202 rated Provider :106 with -1
Consumer :202 evaluates Provider :106 satisfaction: 0.0disatisfaction :1.0
Consumer :202's current satisfaction and disatisfaction for Provider :106 are : 9.0 :3.0
12.0
Bayes Consumer :202 evaluates Provider :106 satisfaction: 0.0disatisfaction :1.0
Consumer :202's current satisfaction and disatisfaction for Provider :106 are : 9.0 :3.0
12.0
Consumer :106 rated Provider :202 with -1
Consumer :106 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :106's current satisfaction and disatisfaction for Provider :202 are : 9.0 :4.0
13.0
Bayes Consumer :106 evaluates Provider :202 satisfaction: 0.0disatisfaction :1.0
Consumer :106's current satisfaction and disatisfaction for Provider :202 are : 9.0 :4.0
13.0
Consumer :202 rated Provider :106 with -1
Consumer :2

In [431]:
#c7,z
for i in range(5):
    c7.interact(z, 1)
    c7.eval_provider(z, 1)
    c7.eval_provider(z, 1, bayes=True)

    z.interact(c7, 1)
    z.eval_provider(c7, 1)
    z.eval_provider(c7, 1, bayes=True)

Consumer :106 rated Provider :204 with 1
Consumer :106 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :106's current satisfaction and disatisfaction for Provider :204 are : 1.0 :0.0
1.0
Bayes Consumer :106 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :106's current satisfaction and disatisfaction for Provider :204 are : 1.0 :0.0
1.0
Consumer :204 rated Provider :106 with 1
Consumer :204 evaluates Provider :106 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :106 are : 1.0 :0.0
1.0
Bayes Consumer :204 evaluates Provider :106 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :106 are : 1.0 :0.0
1.0
Consumer :106 rated Provider :204 with 1
Consumer :106 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :106's current satisfaction and disatisfaction for Provider :204 are : 2.0 :0.0
2.0
Bayes Consumer :106 evalu

In [432]:
c8 = l_consumers[107]
#c8, x
for i in range(15):
    c8.interact(x, 1)
    c8.eval_provider(x, 1)
    c8.eval_provider(x, 1, bayes=True)

    x.interact(c8, 1)
    x.eval_provider(c8, 1)
    x.eval_provider(c8, 1, bayes=True)

Consumer :107 rated Provider :200 with 1
Consumer :107 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :107's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Bayes Consumer :107 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :107's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Consumer :200 rated Provider :107 with 1
Consumer :200 evaluates Provider :107 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :107 are : 1.0 :0.0
1.0
Bayes Consumer :200 evaluates Provider :107 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :107 are : 1.0 :0.0
1.0
Consumer :107 rated Provider :200 with 1
Consumer :107 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :107's current satisfaction and disatisfaction for Provider :200 are : 2.0 :0.0
2.0
Bayes Consumer :107 evalu

In [433]:
#c8,u 
for i in range(9):
    c8.interact(u, -1)
    c8.eval_provider(u, -1)
    c8.eval_provider(u, -1, bayes=True)

    u.interact(c8, -1)
    u.eval_provider(c8, -1)
    u.eval_provider(c8, -1, bayes=True)

c8.interact(u, -1)
c8.eval_provider(u, -1)
c8.eval_provider(u, -1, bayes=True)

u.interact(c8, 1)
u.eval_provider(c8, 1)
u.eval_provider(c8, 1, bayes=True)

Consumer :107 rated Provider :206 with -1
Consumer :107 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :107's current satisfaction and disatisfaction for Provider :206 are : 0.0 :1.0
1.0
Bayes Consumer :107 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :107's current satisfaction and disatisfaction for Provider :206 are : 0.0 :1.0
1.0
Consumer :206 rated Provider :107 with -1
Consumer :206 evaluates Provider :107 satisfaction: 0.0disatisfaction :1.0
Consumer :206's current satisfaction and disatisfaction for Provider :107 are : 0.0 :1.0
1.0
Bayes Consumer :206 evaluates Provider :107 satisfaction: 0.0disatisfaction :1.0
Consumer :206's current satisfaction and disatisfaction for Provider :107 are : 0.0 :1.0
1.0
Consumer :107 rated Provider :206 with -1
Consumer :107 evaluates Provider :206 satisfaction: 0.0disatisfaction :1.0
Consumer :107's current satisfaction and disatisfaction for Provider :206 are : 0.0 :2.0
2.0
Bayes Consumer :107 ev

In [434]:
#c8,v 
for i in range(9):
    c8.interact(v, -1)
    c8.eval_provider(v, -1)
    c8.eval_provider(v, -1, bayes=True)

    v.interact(c8, -1)
    v.eval_provider(c8, -1)
    v.eval_provider(c8, -1, bayes=True)

c8.interact(v, -1)
c8.eval_provider(v, -1)
c8.eval_provider(v, -1, bayes=True)

v.interact(c8, 1)
v.eval_provider(c8, 1)
v.eval_provider(c8, 1, bayes=True)

Consumer :107 rated Provider :208 with -1
Consumer :107 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :107's current satisfaction and disatisfaction for Provider :208 are : 0.0 :1.0
1.0
Bayes Consumer :107 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :107's current satisfaction and disatisfaction for Provider :208 are : 0.0 :1.0
1.0
Consumer :208 rated Provider :107 with -1
Consumer :208 evaluates Provider :107 satisfaction: 0.0disatisfaction :1.0
Consumer :208's current satisfaction and disatisfaction for Provider :107 are : 0.0 :1.0
1.0
Bayes Consumer :208 evaluates Provider :107 satisfaction: 0.0disatisfaction :1.0
Consumer :208's current satisfaction and disatisfaction for Provider :107 are : 0.0 :1.0
1.0
Consumer :107 rated Provider :208 with -1
Consumer :107 evaluates Provider :208 satisfaction: 0.0disatisfaction :1.0
Consumer :107's current satisfaction and disatisfaction for Provider :208 are : 0.0 :2.0
2.0
Bayes Consumer :107 ev

In [435]:
c9 = l_consumers[108]
#c9, x
for i in range(15):
    c9.interact(x, 1)
    c9.eval_provider(x, 1)
    c9.eval_provider(x, 1, bayes=True)

    x.interact(c9, 1)
    x.eval_provider(c9, 1)
    x.eval_provider(c9, 1, bayes=True)

#c9,u 
for i in range(9):
    c9.interact(u, -1)
    c9.eval_provider(u, -1)
    c9.eval_provider(u, -1, bayes=True)

    u.interact(c9, -1)
    u.eval_provider(c9, -1)
    u.eval_provider(c9, -1, bayes=True)

c9.interact(u, -1)
c9.eval_provider(u, -1)
c9.eval_provider(u, -1, bayes=True)

u.interact(c9, 1)
u.eval_provider(c9, 1)
u.eval_provider(c9, 1, bayes=True)


#c9,v 
for i in range(9):
    c9.interact(v, -1)
    c9.eval_provider(v, -1)
    c9.eval_provider(v, -1, bayes=True)

    v.interact(c9, -1)
    v.eval_provider(c9, -1)
    v.eval_provider(c9, -1, bayes=True)

c9.interact(v, -1)
c9.eval_provider(v, -1)
c9.eval_provider(v, -1, bayes=True)

v.interact(c9, 1)
v.eval_provider(c9, 1)
v.eval_provider(c9, 1, bayes=True)

Consumer :108 rated Provider :200 with 1
Consumer :108 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :108's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Bayes Consumer :108 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :108's current satisfaction and disatisfaction for Provider :200 are : 1.0 :0.0
1.0
Consumer :200 rated Provider :108 with 1
Consumer :200 evaluates Provider :108 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :108 are : 1.0 :0.0
1.0
Bayes Consumer :200 evaluates Provider :108 satisfaction: 1.0disatisfaction :0.0
Consumer :200's current satisfaction and disatisfaction for Provider :108 are : 1.0 :0.0
1.0
Consumer :108 rated Provider :200 with 1
Consumer :108 evaluates Provider :200 satisfaction: 1.0disatisfaction :0.0
Consumer :108's current satisfaction and disatisfaction for Provider :200 are : 2.0 :0.0
2.0
Bayes Consumer :108 evalu

In [436]:
c10 = l_consumers[109]
#c10,x
for i in range(2):
    c10.interact(x, -1)
    c10.eval_provider(x, -1)
    c10.eval_provider(x, -1, bayes=True)

    x.interact(c10, -1)
    x.eval_provider(c10, -1)
    x.eval_provider(c10, -1, bayes=True)

Consumer :109 rated Provider :200 with -1
Consumer :109 evaluates Provider :200 satisfaction: 0.0disatisfaction :1.0
Consumer :109's current satisfaction and disatisfaction for Provider :200 are : 0.0 :1.0
1.0
Bayes Consumer :109 evaluates Provider :200 satisfaction: 0.0disatisfaction :1.0
Consumer :109's current satisfaction and disatisfaction for Provider :200 are : 0.0 :1.0
1.0
Consumer :200 rated Provider :109 with -1
Consumer :200 evaluates Provider :109 satisfaction: 0.0disatisfaction :1.0
Consumer :200's current satisfaction and disatisfaction for Provider :109 are : 0.0 :1.0
1.0
Bayes Consumer :200 evaluates Provider :109 satisfaction: 0.0disatisfaction :1.0
Consumer :200's current satisfaction and disatisfaction for Provider :109 are : 0.0 :1.0
1.0
Consumer :109 rated Provider :200 with -1
Consumer :109 evaluates Provider :200 satisfaction: 0.0disatisfaction :1.0
Consumer :109's current satisfaction and disatisfaction for Provider :200 are : 0.0 :2.0
2.0
Bayes Consumer :109 ev

In [437]:
#c10,u   
for i in range(6):
    c10.interact(u, 1)
    c10.eval_provider(u, 1)
    c10.eval_provider(u, 1, bayes=True)

    u.interact(c10, 1)
    u.eval_provider(c10, 1)
    u.eval_provider(c10, 1, bayes=True)

for i in range(3):
    c10.interact(u, -1)
    c10.eval_provider(u, -1)
    c10.eval_provider(u, -1, bayes=True)

    u.interact(c10, -1)
    u.eval_provider(c10, -1)
    u.eval_provider(c10, -1, bayes=True)
    
c10.interact(u, -1)
c10.eval_provider(u, -1)
c10.eval_provider(u, -1, bayes=True)

u.interact(c10, 1)
u.eval_provider(c10, 1)
u.eval_provider(c10, 1, bayes=True)

Consumer :109 rated Provider :206 with 1
Consumer :109 evaluates Provider :206 satisfaction: 1.0disatisfaction :0.0
Consumer :109's current satisfaction and disatisfaction for Provider :206 are : 1.0 :0.0
1.0
Bayes Consumer :109 evaluates Provider :206 satisfaction: 1.0disatisfaction :0.0
Consumer :109's current satisfaction and disatisfaction for Provider :206 are : 1.0 :0.0
1.0
Consumer :206 rated Provider :109 with 1
Consumer :206 evaluates Provider :109 satisfaction: 1.0disatisfaction :0.0
Consumer :206's current satisfaction and disatisfaction for Provider :109 are : 1.0 :0.0
1.0
Bayes Consumer :206 evaluates Provider :109 satisfaction: 1.0disatisfaction :0.0
Consumer :206's current satisfaction and disatisfaction for Provider :109 are : 1.0 :0.0
1.0
Consumer :109 rated Provider :206 with 1
Consumer :109 evaluates Provider :206 satisfaction: 1.0disatisfaction :0.0
Consumer :109's current satisfaction and disatisfaction for Provider :206 are : 2.0 :0.0
2.0
Bayes Consumer :109 evalu

In [438]:
#c10,v   
for i in range(6):
    c10.interact(v, 1)
    c10.eval_provider(v, 1)
    c10.eval_provider(v, 1, bayes=True)

    v.interact(c10, 1)
    v.eval_provider(c10, 1)
    v.eval_provider(c10, 1, bayes=True)

for i in range(5):
    c10.interact(v, -1)
    c10.eval_provider(v, -1)
    c10.eval_provider(v, -1, bayes=True)

    v.interact(c10, -1)
    v.eval_provider(c10, -1)
    v.eval_provider(c10, -1, bayes=True)
    
c10.interact(v, -1)
c10.eval_provider(v, -1)
c10.eval_provider(v, -1, bayes=True)

v.interact(c10, 1)
v.eval_provider(c10, 1)
v.eval_provider(c10, 1, bayes=True)

Consumer :109 rated Provider :208 with 1
Consumer :109 evaluates Provider :208 satisfaction: 1.0disatisfaction :0.0
Consumer :109's current satisfaction and disatisfaction for Provider :208 are : 1.0 :0.0
1.0
Bayes Consumer :109 evaluates Provider :208 satisfaction: 1.0disatisfaction :0.0
Consumer :109's current satisfaction and disatisfaction for Provider :208 are : 1.0 :0.0
1.0
Consumer :208 rated Provider :109 with 1
Consumer :208 evaluates Provider :109 satisfaction: 1.0disatisfaction :0.0
Consumer :208's current satisfaction and disatisfaction for Provider :109 are : 1.0 :0.0
1.0
Bayes Consumer :208 evaluates Provider :109 satisfaction: 1.0disatisfaction :0.0
Consumer :208's current satisfaction and disatisfaction for Provider :109 are : 1.0 :0.0
1.0
Consumer :109 rated Provider :208 with 1
Consumer :109 evaluates Provider :208 satisfaction: 1.0disatisfaction :0.0
Consumer :109's current satisfaction and disatisfaction for Provider :208 are : 2.0 :0.0
2.0
Bayes Consumer :109 evalu

In [439]:
c11 = l_consumers[110]
for i in range(9):
    c11.interact(y, 1)
    c11.eval_provider(y, 1)
    c11.eval_provider(y, 1, bayes=True)

    y.interact(c11, 1)
    y.eval_provider(c11, 1)
    y.eval_provider(c11, 1, bayes=True)

Consumer :110 rated Provider :202 with 1
Consumer :110 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :110's current satisfaction and disatisfaction for Provider :202 are : 1.0 :0.0
1.0
Bayes Consumer :110 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :110's current satisfaction and disatisfaction for Provider :202 are : 1.0 :0.0
1.0
Consumer :202 rated Provider :110 with 1
Consumer :202 evaluates Provider :110 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :110 are : 1.0 :0.0
1.0
Bayes Consumer :202 evaluates Provider :110 satisfaction: 1.0disatisfaction :0.0
Consumer :202's current satisfaction and disatisfaction for Provider :110 are : 1.0 :0.0
1.0
Consumer :110 rated Provider :202 with 1
Consumer :110 evaluates Provider :202 satisfaction: 1.0disatisfaction :0.0
Consumer :110's current satisfaction and disatisfaction for Provider :202 are : 2.0 :0.0
2.0
Bayes Consumer :110 evalu

In [440]:
#c11, z
for i in range(4):
    c11.interact(z, 1)
    c11.eval_provider(z, 1)
    c11.eval_provider(z, 1, bayes=True)

    z.interact(c11, 1)
    z.eval_provider(c11, 1)
    z.eval_provider(c11, 1, bayes=True)
    
for i in range(3):
    c11.interact(z, -1)
    c11.eval_provider(z, -1)
    c11.eval_provider(z, -1, bayes=True)

    z.interact(c11, -1)
    z.eval_provider(c11, -1)
    z.eval_provider(c11, -1, bayes=True)

c11.interact(z, -1)
c11.eval_provider(z, -1)
c11.eval_provider(z, -1, bayes=True)

z.interact(c11, 1)
z.eval_provider(c11, 1)
z.eval_provider(c11, 1, bayes=True)

Consumer :110 rated Provider :204 with 1
Consumer :110 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :110's current satisfaction and disatisfaction for Provider :204 are : 1.0 :0.0
1.0
Bayes Consumer :110 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :110's current satisfaction and disatisfaction for Provider :204 are : 1.0 :0.0
1.0
Consumer :204 rated Provider :110 with 1
Consumer :204 evaluates Provider :110 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :110 are : 1.0 :0.0
1.0
Bayes Consumer :204 evaluates Provider :110 satisfaction: 1.0disatisfaction :0.0
Consumer :204's current satisfaction and disatisfaction for Provider :110 are : 1.0 :0.0
1.0
Consumer :110 rated Provider :204 with 1
Consumer :110 evaluates Provider :204 satisfaction: 1.0disatisfaction :0.0
Consumer :110's current satisfaction and disatisfaction for Provider :204 are : 2.0 :0.0
2.0
Bayes Consumer :110 evalu

In [441]:
# c1 = Consumer(50)
# l_consumers.append(c1)

In [442]:
# p1 = c(100)
# p2 = Producer(101)
# p3 = Producer(102)



# c1.interact(p1, 0.2)
# p1.interact(c1, 0.5)

# c1.interact(p2, 0.4)
# p2.interact(c1, 0.4)

# c1.interact(p3, 0.4)
# p3.interact(c1, 0.4)

In [443]:
# c1.interact(p1, 0.3)
# c1.eval_provider(p1, 0.3)
# c1.eval_provider(p1, 0.3, bayes=True)

In [444]:
# p1.interact(c1, 0.7)
# p1.eval_provider(c1, 0.7)


In [445]:
# p1.eval_provider(c1, 0.7, bayes=True)

In [446]:
# print(str([x.id for x in c1.get_providers()]))

In [447]:
# p1.my_bayes()
#we want to test p =100
# p1 = p1
cum_sat = 0
cum_dis = 0
# print(x.id)
for _, cm in l_consumers.items():
#     print(cm.id)
    s, d = cm.my_bayes(x)
    print('For user = '+str(cm.label)+','+str(cm.id)+' S = '+str(s)+" D = "+str(d))
    
    cum_sat += s
    cum_dis += d
print(str(cum_sat)+'/'+str(cum_sat+ cum_dis))
bayes = (1 + cum_sat) / (2 + (cum_sat+ cum_dis))

For user = C1,100 S = 15.0 D = 0.0
For user = C2,101 S = 15.0 D = 0.0
For user = C3,102 S = 15.0 D = 0.0
For user = C4,103 S = 15.0 D = 0.0
For user = C5,104 S = 0.0 D = 1.0
For user = C6,105 S = 0.0 D = 2.0
For user = C7,106 S = 0.0 D = 3.0
For user = C8,107 S = 15.0 D = 0.0
For user = C9,108 S = 15.0 D = 0.0
For user = C10,109 S = 0.0 D = 2.0
For user = C11,110 S = 0 D = 0
90.0/98.0


In [448]:
bayes

0.91

In [449]:
c1.get_belief(x)

15.0


(0.9375, 0.0, 0.0625)

In [450]:
c2.get_belief(x)

15.0


(0.9375, 0.0, 0.0625)

In [451]:
c3.get_belief(x)

15.0


(0.9375, 0.0, 0.0625)

In [452]:
c4.get_belief(x)

15.0


(0.9375, 0.0, 0.0625)

In [453]:
c5.get_belief(x)

1.0


(0.0, 0.5, 0.5)

In [454]:
c6.get_belief(x)

2.0


(0.0, 0.6666666666666666, 0.3333333333333333)

In [455]:
c10.get_belief(x)

2.0


(0.0, 0.6666666666666666, 0.3333333333333333)

In [456]:
c9.get_belief(x)

15.0


(0.9375, 0.0, 0.0625)

In [457]:
c8.get_belief(x)

15.0


(0.9375, 0.0, 0.0625)

In [458]:
c11.get_belief(y)

9.0


(0.9, 0.0, 0.1)

In [459]:
c11.get_belief(z)

8.0


(0.4444444444444444, 0.4444444444444444, 0.1111111111111111)

In [460]:
c1.get_belief(y)

10.0


(0.0, 0.9090909090909091, 0.09090909090909091)

In [461]:
c2.get_belief(y)

10.0


(0.0, 0.9090909090909091, 0.09090909090909091)

In [462]:
c3.get_belief(y)

10.0


(0.0, 0.9090909090909091, 0.09090909090909091)

In [463]:
c4.get_belief(y)

10.0


(0.0, 0.9090909090909091, 0.09090909090909091)

In [464]:
c1.get_belief(z)

10.0


(0.0, 0.9090909090909091, 0.09090909090909091)

In [465]:
c2.get_belief(z)

10.0


(0.0, 0.9090909090909091, 0.09090909090909091)

In [466]:
c3.get_belief(z)

10.0


(0.0, 0.9090909090909091, 0.09090909090909091)

In [467]:
c4.get_belief(z)

10.0


(0.0, 0.9090909090909091, 0.09090909090909091)

In [468]:
c5.get_belief(y)

6.0


(0.7142857142857143, 0.14285714285714285, 0.14285714285714285)

In [469]:
c5.get_belief(z)

5.0


(0.5, 0.3333333333333333, 0.16666666666666666)

In [470]:
c6.get_belief(y)

6.0


(0.8571428571428571, 0.0, 0.14285714285714285)

In [471]:
c6.get_belief(z)

4.0


(0.4, 0.4, 0.2)

In [472]:
c7.get_belief(y)

14.0


(0.6, 0.3333333333333333, 0.06666666666666667)

In [473]:
c7.get_belief(z)

5.0


(0.8333333333333334, 0.0, 0.16666666666666666)

In [474]:
c11.get_similarity(c1)

9.0
10.0
9.0
10.0
8.0
10.0
8.0
10.0


0.32026912680630204

In [475]:
c11.get_similarity(c2)

9.0
10.0
9.0
10.0
8.0
10.0
8.0
10.0


0.32026912680630204

In [476]:
c11.get_similarity(c3)

9.0
10.0
9.0
10.0
8.0
10.0
8.0
10.0


0.32026912680630204

In [477]:
c11.get_similarity(c4)

9.0
10.0
9.0
10.0
8.0
10.0
8.0
10.0


0.32026912680630204

In [478]:
c11.get_similarity(c5)

9.0
6.0
9.0
6.0
8.0
5.0
8.0
5.0


0.8676744338350197

In [479]:
c11.get_similarity(c6)

9.0
6.0
9.0
6.0
8.0
4.0
8.0
4.0


0.9400814106254535

In [480]:
c11.get_similarity(c7)

9.0
14.0
9.0
14.0
8.0
5.0
8.0
5.0


0.6312925098952105

In [481]:
# c8.get_belief(u)

In [482]:
u.get_belief(c8)

10.0


(0.09090909090909091, 0.8181818181818182, 0.09090909090909091)

In [483]:
# c9.get_belief(u)

In [484]:
u.get_belief(c9)

10.0


(0.09090909090909091, 0.8181818181818182, 0.09090909090909091)

In [485]:
u.get_belief(c10)

10.0


(0.6363636363636364, 0.2727272727272727, 0.09090909090909091)

In [486]:
# c9.get_honesty()

In [487]:
v.get_belief(c8)

10.0


(0.09090909090909091, 0.8181818181818182, 0.09090909090909091)

In [488]:
v.get_belief(c9)

10.0


(0.09090909090909091, 0.8181818181818182, 0.09090909090909091)

In [489]:
v.get_belief(c10)

12.0


(0.5384615384615384, 0.38461538461538464, 0.07692307692307693)

In [490]:
x.get_belief(c8)

15.0


(0.9375, 0.0, 0.0625)

In [491]:
x.get_belief(c9)

15.0


(0.9375, 0.0, 0.0625)

In [492]:
x.get_belief(c10)

2.0


(0.0, 0.6666666666666666, 0.3333333333333333)

In [493]:
#  c8.get_honesty()

In [494]:
c8.get_global_crediblity()

15.0
10.0
10.0


0.37310606060606055

In [495]:
c9.get_global_crediblity()

15.0
10.0
10.0


0.37310606060606055

In [496]:
c10.get_global_crediblity()

2.0
10.0
12.0


0.3916083916083916

In [497]:
all_v = [c1,c2,c3,c4,c5,c6,c7,c8,c9,c10]
all_p = [x,y,z,u,v]

In [498]:
# all_v
all_p

In [499]:
c11.p_g_reputation(x, all_v)

15.0
9.0
10.0
9.0
10.0
8.0
10.0
8.0
10.0
15.0
9.0
10.0
9.0
10.0
8.0
10.0
8.0
10.0
15.0
9.0
10.0
9.0
10.0
8.0
10.0
8.0
10.0
15.0
9.0
10.0
9.0
10.0
8.0
10.0
8.0
10.0
1.0
9.0
6.0
9.0
6.0
8.0
5.0
8.0
5.0
2.0
9.0
6.0
9.0
6.0
8.0
4.0
8.0
4.0
3.0
9.0
14.0
9.0
14.0
8.0
5.0
8.0
5.0
15.0
15.0
10.0
10.0
15.0
15.0
10.0
10.0
2.0
2.0
10.0
12.0
personilsed_reputaion: 0.0 g_rep: 0.6995738636363635


0.04663825757575756

In [514]:
def mod_BRS(provider, all_advisors, all_providers, i_S = 0.5, i_D = 0.5):
    rep = 0
    reputation = 0
    advisor_count = 0
    for a in all_advisors:
        if provider in a.b_satisfaction:
            advisor_count += 1
            e_p = (i_S + a.b_satisfaction[provider])/ (i_S + i_D + a.b_satisfaction[provider] + a.b_disatisfaction[provider])
            a_cred = 0
            sum_sat = 0
            sum_dis = 0
            for p in all_providers:
                if a in p.b_satisfaction:
                    print(str(p.id)+' evaluated '+str(a.label)+'sat '+str(p.b_satisfaction[a])+'dis '+str(p.b_disatisfaction[a]))
#                     print(str(p.id)+' evaluated ''sat '+str(a.disatisfaction[p])+'dis '+str(p.disatisfaction[a]))
#                     print(a.disatisfaction[p])
#                     print('sat '+str(p.satisfaction[a]))
#                     print('dis '+str(p.disatisfaction[a]))
                    sum_sat += p.b_satisfaction[a]
                    sum_dis += p.b_disatisfaction[a]
#                     sum_sat += a.disatisfaction[p]
#             print('sum_sat ='+str(sum_sat)+' sum_dis ='+str(sum_dis))
            if (sum_sat+sum_dis) > 0:
                a_cred = 1 - (sum_dis/(sum_sat+sum_dis))
            
#             print('a '+str(a.label)+' for provider '+str(p.id)+'e_p :'+str(e_p))
            print('a '+str(a.label)+' a_cred :'+str(a_cred))
            
            rep += (e_p * a_cred)
    if advisor_count > 0:
        reputation = rep / advisor_count
    return reputation
    

In [515]:
mod_BRS(x, all_v, all_p)

200 evaluated C1sat 15.0dis 0.0
202 evaluated C1sat 1.0dis 9.0
204 evaluated C1sat 2.0dis 8.0
206 evaluated C1sat 1.0dis 9.0
208 evaluated C1sat 2.0dis 8.0
a C1 a_cred :0.38181818181818183
200 evaluated C2sat 15.0dis 0.0
202 evaluated C2sat 1.0dis 9.0
204 evaluated C2sat 1.0dis 9.0
206 evaluated C2sat 2.0dis 8.0
208 evaluated C2sat 2.0dis 8.0
a C2 a_cred :0.38181818181818183
200 evaluated C3sat 15.0dis 0.0
202 evaluated C3sat 2.0dis 8.0
204 evaluated C3sat 2.0dis 8.0
206 evaluated C3sat 1.0dis 9.0
208 evaluated C3sat 2.0dis 8.0
a C3 a_cred :0.4
200 evaluated C4sat 15.0dis 0.0
202 evaluated C4sat 1.0dis 9.0
204 evaluated C4sat 1.0dis 9.0
206 evaluated C4sat 1.0dis 9.0
208 evaluated C4sat 2.0dis 8.0
a C4 a_cred :0.36363636363636365
200 evaluated C5sat 0.0dis 1.0
202 evaluated C5sat 6.0dis 0.0
204 evaluated C5sat 4.0dis 1.0
a C5 a_cred :0.8333333333333334
200 evaluated C6sat 0.0dis 2.0
202 evaluated C6sat 6.0dis 0.0
204 evaluated C6sat 3.0dis 1.0
a C6 a_cred :0.75
200 evaluated C7sat 0.0d

0.2936399711399712

In [502]:
import random

class Experiment:
    def __init__(self, scenario, nconsumer, nproducer, ntransactions, seed=100):
        self.scenario = scenario
        self.nconsumer = nconsumer
        self.nproducer = nproducer
        self.ntransactions = ntransactions
        self.nfaket = int(ntransactions/10)
        self.init_trans = ntransactions - self.nfaket
        self.consumers = {}
        self.producers = {}
        random.seed(seed)
        self.graphdata = []
        self.graphdata2 = []
        self.badmouthdata = []
        self.badmouthdata2 = []
        self.start()
        
        
    def start(self):
        consumers = {}
        producers = {}
        consumer_id_start_range = 100
        producer_id_start_range = 200
        for x in range(self.nconsumer):
            c = Consumer(consumer_id_start_range+x)
            consumers[c.id] = c
        for y in range(self.nproducer):
            p = Consumer(producer_id_start_range+y)
#             producers.append(p)
            producers[p.id] = p
        print('experiment'+str(self.scenario)+' initialised with'+str(len(consumers))+" c and "+str(len(producers))+" p")
        self.consumers = consumers
        self.producers = producers
        
        targetc = consumers[103]
        targetp = producers[202]
        for _,c in consumers.items():
            for i in range(self.init_trans):
#                 
                _, p = random.choice(list(producers.items()))
                c.interact(p, random.uniform(0, 1))
                p.interact(c, random.uniform(0, 1))
                if c == targetc and p == targetp:
                    self.graphdata.append(targetc.reputation(targetp))
                    self.graphdata2.append(targetc.bayes(targetp))
        

    def ballot_stuffing(self, providerid, consumerid, rating=0.9, targetc=103, n_consumers=1):
        p = self.producers[providerid]
        c = self.consumers[consumerid]
        c2 = self.consumers[consumerid+1]
        t = self.consumers[targetc]
        for x in range(self.nfaket):
            c.interact(p, rating)
            p.interact(c, rating)
            c2.interact(p, rating)
            p.interact(c2, rating)
            self.graphdata.append(t.reputation(p))
            self.graphdata2.append(t.bayes(p))
            
    def bad_mouthing(self, victim, consumerid, rating=0.001, targetc=103, n_consumers=1):
        p = self.producers[victim]
        c = self.consumers[consumerid]
        c2 = self.consumers[consumerid+1]
        t = self.consumers[targetc]
        for x in range(self.nfaket):
            c.interact(p, rating)
            p.interact(c, random.uniform(0, 1))
            c2.interact(p, rating)
            p.interact(c2, random.uniform(0, 1))
            self.graphdata.append(t.reputation(p))
            self.graphdata2.append(t.bayes(p))
            
    def majority_descrimination(self, attacker, consumerid, rating=0.001, targetc=103, n_consumers=1):
        p = self.producers[attacker]
        t = self.consumers[targetc]
        for _, c in self.consumers.items():
            if c != t:
                for x in range(self.nfaket):
                    if c.id <= 104:
                        c.interact(p, 1)
                        p.interact(c, 1)
                    else:
                        c.interact(p, random.uniform(0, 0.5))
                        p.interact(c, 0)
                    self.graphdata.append(t.reputation(p))
                    self.graphdata2.append(t.bayes(p))
                                   
    def minority_descrimination(self, attacker, consumerid, rating=0.001, targetc=103, n_consumers=1):
        p = self.producers[attacker]
        t = self.consumers[targetc]
        for _, c in self.consumers.items():
            if c != t:
                for x in range(self.nfaket):
                    if c.id <= 104:
                        c.interact(p, 0)
                        p.interact(c, 0)
                    else:
                        c.interact(p, random.uniform(0.5, 1))
                        p.interact(c, random.uniform(0.5, 1))
                    self.graphdata.append(t.reputation(p))
                    self.graphdata2.append(t.bayes(p))
        
    
    def plot(self):
#         import matplotlib.pyplot as plt
#         xlables = [x for x in range(0, self.init_trans,10)]
#         fig = plt.figure()
#         ax = fig.add_subplot(111)
#         ax.plot([1, 2, 3, 4,5,6,7,8,9], xlables, color='lightblue', linewidth=3)
#         ax.scatter([0.3, 3.8, 1.2, 2.5, 2.5, 2.6, 2.7,3.0,3.5 ], [11, 25, 9,5,30,33,25, 26,35], color='darkgreen', marker='^')
#         ax.set_xlim(0.5, 4.5)
#         plt.show()
        import matplotlib.pyplot as plt
        import numpy as np
        
        print(self.graphdata)

        # Prepare the data
        x = np.linspace(0, len(self.graphdata), len(self.graphdata))

        # Plot the data
        plt.plot(x, self.graphdata2, alpha=0.7, label='Bayes')
        plt.plot(x, self.graphdata, alpha=0.7, label='ROBUSTRA')

        # Add a legend
        plt.legend()

        # Show the plot
        plt.show()
        